In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from matplotlib import pyplot as plt
import seaborn as sns

In [2]:
# File paths and separator
DATA_PATH_stages = "../data/extracted/kdigo_stages_measured.csv"
# DATA_PATH_labs = "../data/extracted/labs-kdigo_stages_measured.csv"
DATA_PATH_labs_new = "../data/extracted/labs_new.csv"
# DATA_PATH_vitals = "../data/extracted/vitals-kdigo_stages_measured.csv"
DATA_PATH_vitals = "../data/extracted/vitals.csv"
# DATA_PATH_vents = "../data/extracted/vents-vasopressor-sedatives-kdigo_stages_measured.csv"
DATA_PATH_vents = "../data/extracted/vents_vasopressor_sedatives.csv"
# DATA_PATH_detail = "../data/extracted/icustay_detail-kdigo_stages_measured.csv"
DATA_PATH_detail = "../data/extracted/icustay_detail.csv"
DATA_PATH_heightweight = "../data/extracted/heightweight.csv"
DATA_PATH_calcium = "../data/extracted/calcium.csv"
DATA_PATH_inr_max = "../data/extracted/inr_max.csv"
SEPARATOR = ";"

# Constants
IMPUTE_EACH_ID = False
IMPUTE_COLUMN = False
TESTING = False
TEST_SIZE = 0.05
SPLIT_SIZE = 0.2
MAX_DAYS = 35
CLASS1 = True
ALL_STAGES = False
MAX_FEATURE_SET = True
FIRST_TURN_POS = True
TIME_SAMPLING = True
SAMPLING_INTERVAL = '6H'
RESAMPLE_LIMIT = 16
MOST_COMMON = False
IMPUTE_METHOD = 'most_frequent'
FILL_VALUE = 0
ADULTS_MIN_AGE = 18
ADULTS_MAX_AGE = 120
NORMALIZATION = 'min-max'
HOURS_AHEAD = 48
NORM_TYPE = 'min_max'
RANDOM = 42

def filter_by_length_of_stay(X):
    drop_list = []
    long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)

    for icustay_id, is_long in long_stays.items():
        if is_long:
            max_time = X[X['icustay_id'] == icustay_id]['charttime'].max() - pd.to_timedelta(MAX_DAYS, unit='D')
            X = X[~((X['icustay_id'] == icustay_id) & (X['charttime'] < max_time))]

    short_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) < (HOURS_AHEAD/24))
    drop_list = short_stays[short_stays].index.tolist()

    X = X[~X.icustay_id.isin(drop_list)]
    return X

In [48]:
# Load datasets
print("Loading datasets...")
X = pd.read_csv(DATA_PATH_stages, sep=SEPARATOR)
X.drop(["aki_stage_creat", "aki_stage_uo"], axis=1, inplace=True)
X = X.dropna(how='all', subset=['creat', 'uo_rt_6hr', 'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage'])
X['charttime'] = pd.to_datetime(X['charttime'])

print(len(X))
print(X['aki_stage'].value_counts())

dataset_detail = pd.read_csv(DATA_PATH_detail, sep=SEPARATOR)
dataset_detail.drop(['dod', 'admittime', 'dischtime', 'los_hospital', 'ethnicity', 
                     'hospital_expire_flag', 'hospstay_seq', 'first_hosp_stay', 'intime', 
                     'outtime', 'los_icu', 'icustay_seq', 'first_icu_stay'], axis=1, inplace=True)

# dataset_labs = pd.read_csv(DATA_PATH_labs, sep=SEPARATOR)
# dataset_labs = dataset_labs.dropna(subset=['charttime']).dropna(subset=dataset_labs.columns[4:], how='all')
# dataset_labs['charttime'] = pd.to_datetime(dataset_labs['charttime'])
# dataset_labs = dataset_labs.sort_values(by=['icustay_id', 'charttime'])
# dataset_labs.drop(['albumin_min', 'albumin_max','bilirubin_min', 'bilirubin_max','bands_min', 'bands_max',
#                    'lactate_min', 'lactate_max','platelet_min', 'platelet_max','ptt_min', 'ptt_max', 
#                    'inr_min', 'inr_max', 'pt_min', 'pt_max'], axis = 1, inplace = True)

dataset_labs_new = pd.read_csv(DATA_PATH_labs_new, sep=SEPARATOR)
dataset_labs_new = dataset_labs_new.dropna(subset=['charttime']).dropna(subset=dataset_labs_new.columns[4:], how='all')
dataset_labs_new['charttime'] = pd.to_datetime(dataset_labs_new['charttime'])
dataset_labs_new = dataset_labs_new.sort_values(by=['icustay_id', 'charttime'])

dataset_vitals = pd.read_csv(DATA_PATH_vitals, sep=SEPARATOR)
dataset_vents = pd.read_csv(DATA_PATH_vents, sep=SEPARATOR)
dataset_vitals.drop(["heartrate_min", "heartrate_max", "sysbp_min", "sysbp_max", "diasbp_min", "diasbp_max",
                        'meanbp_min', 'meanbp_max', 'tempc_min', 'tempc_max', "resprate_min", "resprate_max", 
                        "spo2_min", "spo2_max", "glucose_min", "glucose_max"], axis=1, inplace=True)
dataset_vitals['charttime'] = pd.to_datetime(dataset_vitals['charttime'])
dataset_vents['charttime'] = pd.to_datetime(dataset_vents['charttime'])
dataset_vitals = dataset_vitals.dropna(subset=dataset_vitals.columns[4:], how='all')
dataset_vitals = dataset_vitals.sort_values(by=['icustay_id', 'charttime'])
dataset_vents = dataset_vents.sort_values(by=['icustay_id', 'charttime'])

dataset_heightweight = pd.read_csv(DATA_PATH_heightweight, sep=SEPARATOR)
dataset_heightweight = dataset_heightweight.dropna(subset=['icustay_id', 'height_first', 'weight_first'], how='all')
dataset_heightweight = dataset_heightweight.sort_values(by=['icustay_id'])

dataset_calcium = pd.read_csv(DATA_PATH_calcium, sep=SEPARATOR)
dataset_calcium.drop(["hadm_id"], axis=1, inplace=True)
dataset_calcium['charttime'] = pd.to_datetime(dataset_calcium['charttime'])
dataset_calcium = dataset_calcium.sort_values(by=['icustay_id', 'charttime'])

dataset_inr_max = pd.read_csv(DATA_PATH_inr_max, sep=SEPARATOR)
dataset_inr_max.drop(["hadm_id", "subject_id"], axis=1, inplace=True)
dataset_inr_max = dataset_inr_max.sort_values(by=['icustay_id'])

# Calculate mean for each pair and drop original columns
# column_pairs = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
#                 ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
#                 ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
#                 ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
#                 ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
#                 ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
#                 ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
#                 ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
#                 ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max')]

# for min_col, max_col in column_pairs:
#     mean_col = min_col.rsplit('_', 1)[0] + '_mean'
#     dataset_labs[mean_col] = dataset_labs[[min_col, max_col]].mean(axis=1)
#     dataset_labs.drop([min_col, max_col], axis=1, inplace=True)

# column_pairs_new = [('aniongap_min', 'aniongap_max'), ('albumin_min', 'albumin_max'), 
#                 ('bands_min', 'bands_max'), ('bicarbonate_min', 'bicarbonate_max'), 
#                 ('bilirubin_min', 'bilirubin_max'), ('creatinine_min', 'creatinine_max'), 
#                 ('chloride_min', 'chloride_max'), ('glucose_min', 'glucose_max'), 
#                 ('hematocrit_min', 'hematocrit_max'), ('hemoglobin_min', 'hemoglobin_max'), 
#                 ('lactate_min', 'lactate_max'), ('platelet_min', 'platelet_max'), 
#                 ('potassium_min', 'potassium_max'), ('ptt_min', 'ptt_max'), 
#                 ('inr_min', 'inr_max'), ('pt_min', 'pt_max'), ('sodium_min', 'sodium_max'), 
#                 ('bun_min', 'bun_max'), ('wbc_min', 'wbc_max'), ('bilirubin_total_min', 'bilirubin_total_max'), 
#                 ('estimated_gfr_min', 'estimated_gfr_max'), ('phosphate_min', 'phosphate_max'), 
#                 ('urea_nitrogen_min', 'urea_nitrogen_max'), ('uric_acid_min', 'uric_acid_max'), 
#                 ('calcium_total_min', 'calcium_total_max'), ('inr_pt_min', 'inr_pt_max'), 
#                 ('platelet_count_min', 'platelet_count_max')]

# for min_col, max_col in column_pairs_new:
#     mean_col = min_col.rsplit('_', 1)[0] + '_mean'
#     dataset_labs_new[mean_col] = dataset_labs_new[[min_col, max_col]].mean(axis=1)
#     dataset_labs_new.drop([min_col, max_col], axis=1, inplace=True)

# Merge datasets
if MAX_FEATURE_SET:
    # X = X.merge(dataset_labs, on=["icustay_id", "charttime"], how="outer")
    X = X.merge(dataset_labs_new, on=["icustay_id", "charttime"], how="outer")
    X = X.merge(dataset_vitals, on=["icustay_id", "charttime", "subject_id", "hadm_id"], how="outer")
    X = X.merge(dataset_vents, on=["icustay_id", "charttime"], how="outer")
    X.drop(["subject_id"], axis=1, inplace=True)
    X = X.merge(dataset_calcium, on=["icustay_id", "charttime"], how="outer")

Loading datasets...
3737147
aki_stage
0    3000795
2     353266
3     207759
1     175327
Name: count, dtype: int64


MemoryError: Unable to allocate 516. MiB for an array with shape (5, 13521169) and data type float64

In [5]:
# take only head of X
X = X.head(50000)

In [22]:
print("Filtering patients by age and length of stay...")
# Filtering patients by age and length of stay
dataset_detail = dataset_detail[dataset_detail['admission_age'] >= ADULTS_MIN_AGE]
adults_icustay_id_list = dataset_detail['icustay_id'].unique()
X = X[X.icustay_id.isin(adults_icustay_id_list)].sort_values(by=['icustay_id', 'charttime'])

X = filter_by_length_of_stay(X)
dataset_detail = dataset_detail[dataset_detail.icustay_id.isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])

Filtering patients by age and length of stay...


C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\1829962070.py:42: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  long_stays = X.groupby(['icustay_id']).apply(lambda group: (group['charttime'].max() - group['charttime'].min()).total_seconds() / (24 * 60 * 60) > MAX_DAYS)
C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\1829962070.py:49: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  short_stays = X.groupby(['

In [23]:
label = ['aki_stage']
skip = ['icustay_id', 'charttime', 'aki_stage']
discrete_feat = ['sedative', 'vasopressor', 'vent', 'hadm_id']
skip.extend(discrete_feat)    
numeric_feat = list(X.columns.difference(skip))

In [43]:
# save to csv
X.to_csv('../data/analysis/data_preprocessed_extended_before_resampling.csv', index=False)

In [24]:
X_old = X.copy()

In [42]:
X = X_old.copy()

In [40]:
SAMPLING_INTERVAL = '1H'


In [46]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
X.describe()

,icustay_id,charttime,creat,uo_rt_6hr,uo_rt_12hr,uo_rt_24hr,aki_stage,hadm_id,aniongap_mean,albumin_mean,bands_mean,bicarbonate_mean,bilirubin_mean,creatinine_mean,chloride_mean,glucose_mean_x,hematocrit_mean,hemoglobin_mean,lactate_mean,platelet_mean,potassium_mean,ptt_mean,inr_mean,pt_mean,sodium_mean,bun_mean,wbc_mean,bilirubin_total_mean,estimated_gfr_mean,phosphate_mean,urea_nitrogen_mean,uric_acid_mean,calcium_total_mean,inr_pt_mean,platelet_count_mean,heartrate_mean,sysbp_mean,diasbp_mean,meanbp_mean,resprate_mean,tempc_mean,spo2_mean,glucose_mean_y,vent,vasopressor,sedative,subject_id,calcium
count,1.089354e+07,10883733,356403.000000,2.927605e+06,2.927605e+06,2.927605e+06,3.357429e+06,7.836421e+06,634480.000000,105565.000000,24372.000000,645102.000000,0.0,660019.000000,704724.000000,835236.000000,814509.000000,691461.000000,182274.000000,0.0,891211.000000,431543.000000,404373.000000,404210.000000,739052.000000,0.0,606226.000000,161149.000000,0.0,525388.000000,657577.000000,10787.000000,519422.000000,404373.000000,627226.000000,5.760386e+06,5.322615e+06,5.321018e+06,5.337530e+06,5.792045e+06,1.591271e+06,5.613001e+06,1.156737e+06,1.083213e+07,1.083213e+07,1.083213e+07,210164.000000,84127.000000
mean,2.504161e+05,2152-01-26 05:46:07.862238208,1.525335,1.417491e+00,1.351479e+00,1.331286e+00,4.130869e-01,1.498615e+05,13.645923,2.921914,6.924680,25.452700,NaN,1.545150,103.748875,133.694788,30.334720,10.237794,2.544478,NaN,4.128352,45.145643,1.621220,16.773095,138.409198,NaN,11.241903,3.736320,NaN,3.582060,30.798039,5.179364,8.419167,1.621220,237.966543,8.693463e+01,1.215397e+02,6.020513e+01,7.910966e+01,2.019648e+01,3.700343e+01,9.707629e+01,1.426761e+02,4.252274e-01,1.720059e-01,2.466332e-01,33776.498768,1.138133
min,2.000010e+05,2100-06-08 22:45:00,0.000000,-6.637170e+01,-6.637170e+01,-4.694840e+01,0.000000e+00,1.000010e+05,1.000000,0.900000,0.800000,2.000000,NaN,0.050000,1.600000,0.139000,2.000000,1.200000,0.050000,NaN,0.600000,0.150000,0.100000,2.390000,1.070000,NaN,0.100000,0.100000,NaN,0.100000,1.000000,0.100000,0.300000,0.100000,4.000000,3.500000e-01,2.000000e-01,4.100000e-01,4.300000e-01,1.700000e-01,1.500000e+01,5.000000e-01,1.060000e-01,0.000000e+00,0.000000e+00,0.000000e+00,3.000000,0.090000
25%,2.254900e+05,2127-06-11 13:00:00,0.700000,5.842000e-01,6.134000e-01,6.492000e-01,0.000000e+00,1.253680e+05,11.000000,2.400000,2.000000,22.000000,NaN,0.700000,100.000000,101.000000,26.900000,9.000000,1.200000,NaN,3.700000,28.600000,1.200000,13.200000,135.000000,NaN,7.100000,0.500000,NaN,2.700000,14.000000,2.800000,7.900000,1.200000,132.000000,7.400000e+01,1.040000e+02,5.000000e+01,6.800000e+01,1.600000e+01,3.644444e+01,9.600000e+01,1.070000e+02,0.000000e+00,0.000000e+00,0.000000e+00,12014.000000,1.070000
50%,2.506990e+05,2152-08-11 01:00:00,1.000000,1.020400e+00,1.036100e+00,1.058000e+00,0.000000e+00,1.491450e+05,13.000000,2.900000,4.000000,25.000000,NaN,1.000000,104.000000,121.000000,29.800000,10.000000,1.800000,NaN,4.100000,35.200000,1.300000,14.500000,138.000000,NaN,9.900000,1.000000,NaN,3.400000,23.000000,4.400000,8.400000,1.300000,209.000000,8.600000e+01,1.190000e+02,5.900000e+01,7.700000e+01,2.000000e+01,3.700000e+01,9.800000e+01,1.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,24084.000000,1.140000
75%,2.754240e+05,2177-03-18 21:00:00,1.600000,1.761900e+00,1.680800e+00,1.640000e+00,0.000000e+00,1.750220e+05,15.000000,3.400000,9.000000,28.000000,NaN,1.700000,108.000000,150.000000,33.200000,11.300000,2.900000,NaN,4.400000,53.400000,1.700000,17.500000,141.000000,NaN,13.600000,3.200000,NaN,4.100000,39.000000,6.800000,8.900000,1.700000,308.000000,9.800000e+01,1.370000e+02,6.900000e+01,8.833330e+01,2.400000e+01,3.755555e+01,9.900000e+01,1.600000e+02,1.000000e+00,0.000000e+00,0.000000e+00,54082.000000,1.190000
max,2.999990e+05,2210-08-24 05:53:00,138.000000,9.000000e+02,9.000000e+02,9.000000e+02,3.000000e+00,1.999990e+05,77.000000,6.900000,86.000000,65.000000,NaN,138.000000,198.000000,3565.000000,77.700000,43.000000,36.

In [41]:
# Resampling
if TIME_SAMPLING:
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max()
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()



C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\3895676415.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


KeyboardInterrupt: 

In [ ]:
# Resampling
if TIME_SAMPLING:
    
    # Set index and group by 'icustay_id' before resampling
    X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)
    
    # Resample and aggregate features
    if MAX_FEATURE_SET:
        X_discrete = X[discrete_feat].max().fillna(FILL_VALUE).astype(np.int64)
    X_numeric = X[numeric_feat].mean()
    X_label = X['aki_stage'].max()

    print("Merging sampled features")
    try:
        X = pd.concat([X_numeric, X_discrete, X_label], axis=1).reset_index()
    except:
        X = pd.concat([X_numeric, X_label], axis=1).reset_index()



# Forward fill again after resampling
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].ffill(limit=RESAMPLE_LIMIT).fillna(0)



# Ensure binary values (convert any positive number to 1)
X['aki_stage'] = (X['aki_stage'] > 0).astype(int)

# Shifting labels
shift_steps = HOURS_AHEAD // int(SAMPLING_INTERVAL[:-1])
X['aki_stage'] = X.groupby('icustay_id')['aki_stage'].shift(-shift_steps)
X = X.dropna(subset=['aki_stage'])

# Merging not time-dependent data
dataset_detail = dataset_detail[dataset_detail['icustay_id'].isin(X['icustay_id'].unique())].sort_values(by=['icustay_id'])
dataset_detail = pd.get_dummies(dataset_detail, columns=['gender', 'ethnicity_grouped'])
dataset_detail.drop(['subject_id', 'hadm_id'], axis=1, inplace=True)
X = X.merge(dataset_detail, on='icustay_id')
X = X.merge(dataset_heightweight, on='icustay_id')
X = X.merge(dataset_inr_max, on='icustay_id')

# If no imputation method selected or only impute each id, for the remaining nan impute direclty with FILL_VALUE
X = X.fillna(FILL_VALUE) 


C:\Users\derqu\AppData\Local\Temp\ipykernel_8124\2742523973.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  X = X.set_index('charttime').groupby('icustay_id').resample(SAMPLING_INTERVAL)


Merging sampled features


In [15]:

# Save preprocessed data
X.to_csv('../data/preprocessed/preprocessed_data_extended_small.csv', index=False)